## ETL

### Extract Data

In [ ]:
import pandas as pd

def extract(url: str) -> pd.DataFrame:
    return pd.read_csv(url)

### Transform Data

In [ ]:
from typing import Dict


def transform(df: pd.DataFrame) -> Dict[str, pd.DataFrame]:
    df["employer_id"] = df["Employer"].astype("category").cat.codes
    df["sector_id"] = df["Sector"].astype("category").cat.codes
    df["job_title_id"] = df["Job Title"].astype("category").cat.codes

    employee_df = df[
        [
            "First Name",
            "Last Name",
            "Salary",
            "Benefits",
            "employer_id",
            "sector_id",
            "job_title_id",
        ]
    ].rename(
        columns={
            "First Name": "first_name",
            "Last Name": "last_name",
            "Salary": "salary",
            "Benefits": "benefits",
        }
    )

    employer_df = (
        df[["employer_id", "Employer"]]
        .rename(columns={"employer_id": "id", "Employer": "name"})
        .drop_duplicates()
    )

    job_title_df = (
        df[["Job Title", "job_title_id"]]
        .rename(columns={"Job Title": "job_title", "job_title_id": "id"})
        .drop_duplicates()
    )

    sector_df = (
        df[["Sector", "sector_id"]]
        .rename(columns={"Sector": "sector", "sector_id": "id"})
        .drop_duplicates()
    )

    result = {"employee": employee_df, "employer": employer_df, "job_title": job_title_df, "sector": sector_df}
    return result

### Load Data

In [ ]:
from sqlalchemy import create_engine

def load(db_con_string: str, table_to_df: Dict[str, pd.DataFrame]):
    engine = create_engine(db_con_string, echo=False)

    with engine.connect() as con:
        for table_name, df in table_to_df.items():
            df.to_sql(table_name, con)